In [2]:
from importlib import reload
import logging

import pandas as pd

import import_excel, views

reload(logging)
reload(import_excel)
reload(views)

logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s %(levelname)s:%(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S')
pd.set_option('display.max_rows', 1000)

view = views.UFNSView()


/Users/maksimtunev/PycharmProjects/balanceTax/venv/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
2023-04-14 10:58:54 INFO:Загрузка файла завершена. В платежах содержится 65 операций.
2023-04-14 10:58:54 INFO:Загрузка файла завершена. В платежах содержится 326 операций.
/Users/maksimtunev/PycharmProjects/balanceTax/venv/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
2023-04-14 10:58:54 INFO:Загрузка файла завершена. В платежах содержится 637 операций.
2023-04-14 10:58:56 INFO:Загрузка файла завершена. В балансе содержится 816 операций.
2023-04-14 10:58:58 INFO:Загрузка файла завершена. В балансе содержится 1729 операций.
2023-04-14 10:59:00 INFO:Загрузка файла завершена

In [154]:
view.current_balance(payments=3543917.07, accrual=3200000)

-2852943.1999999997

In [155]:
view.model_balance()

,balance,credit,credit_correction,debit,debit_penni,debit_penni_291
Прибыль,-15642.00,0.00,0.00,15642.00,0.00,0.00
Страховые - Нетрудоспособность и Материнство,707856.63,1622335.98,950740.63,914479.35,44149.84,0.00
НДС,-182018.88,247477.95,0.00,668885.96,258662.96,239389.13
Страховые - ОМС (до до 1 января 2017),73.83,73.83,0.00,0.00,0.00,0.00
Страховые - Нетрудоспособность и Материнство (до 1 января 2017),0.00,0.00,0.00,0.00,0.00,0.00
Страховые - ОМС,-869350.08,998498.23,36251.52,2064908.35,305414.31,197060.04
Страховые - ПФР,1530429.06,8997857.89,4119553.08,7816725.36,740485.78,349296.53
НДФЛ,-4350392.96,3182648.80,0.00,7720061.53,978972.35,187019.77
Страховые - Доп. Тариф,-15055.87,23415.11,0.00,42929.28,6523.05,4458.30
Прибыль в ФЕД. Бюджет,-2760.00,0.00,0.00,2760.00,0.00,0.00


In [157]:
view.current_balance(payments=3543917.07, accrual=3200000)

-2852943.1999999997

In [147]:
# Проверить пени по НДФЛ попробовать найти оспоримое

df = view.df
mask = (df.operation_details == 'начислены пени (по расчету)') & (df.operation_date > pd.to_datetime('2021-06-30').date())
mask = mask & (df.deadline == pd.to_datetime('2021-06-30').date())
df[mask][view.COLUMNS_READABLE].sort_values(by='deadline').debit.sum()

977226.4900000001

In [168]:
# Что вообще произошло 09-22-2022

df = view.df
mask = (df.operation_date == pd.to_datetime('2022-09-22').date())
df = df[mask][view.COLUMNS_READABLE]
df.reset_index().groupby(by=['tax', 'payment_type', 'operation_details',  'index'])['debit'].sum().sort_index()

# Восстановили 9663903.18 вместе с новыми пенями

tax                                           payment_type  operation_details            index
НДС                                           Налог         списана задолженность        1498     1065299.59
                                              Пеня          начислены пени (по расчету)  1478      239389.13
                                                            списана задолженность        1479      231192.79
                                                                                         1482       44811.06
НДФЛ                                          Налог         списана задолженность        1477      880861.09
                                              Пеня          начислены пени (по расчету)  1485      187019.77
                                                            списана задолженность        1486      309477.80
                                              Штраф         списана задолженность        1489      356344.78
Прибыль                          

In [142]:
df = view.correction_sum()
df.values.sum()

5106545.23

In [172]:
# Что вообще произошло 30-06-2022

df = view.df
mask = (df.operation_date == pd.to_datetime('2021-06-30').date())
df[mask][view.COLUMNS_READABLE].reset_index().groupby(by=['tax', 'payment_type', 'operation_details',  'index'])['credit'].sum().sort_index()



tax                                           payment_type  operation_details            index
НДС                                           Налог         списана задолженность        1765     1065299.59
                                              Пеня          начислены пени (по расчету)  1822           0.00
                                                            списана задолженность        1823      231192.79
                                                                                         1824       44811.06
НДФЛ                                          Налог         списана задолженность        1911      880861.09
                                              Пеня          начислены пени (по расчету)  1951           0.00
                                                            списана задолженность        1952      309477.80
                                              Штраф         списана задолженность        1986      356344.78
Прибыль                          

In [159]:
view.correction_sum()

tax_period,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30,2020-10-31,2020-11-30,2020-12-31,2021-01-31,...,2021-06-30,2021-07-31,2021-08-31,2021-09-30,2021-10-31,2021-11-30,2021-12-31,2022-01-31,2022-02-28,2022-03-31
tax,,,,,,,,,,,,,,,,,,,,,
Страховые - Доп. Тариф,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Страховые - Нетрудоспособность и Материнство,44139.16,48316.08,40350.29,44802.53,37572.72,39824.06,41313.69,35324.86,36978.51,42423.16,...,40465.52,48647.57,36816.79,40148.70,36347.80,34610.06,34799.30,49372.02,33702.67,36077.33
Страховые - ОМС,1527.09,1671.09,1413.13,1544.92,1330.67,1480.35,1638.02,1320.73,1388.57,1828.11,...,1584.83,1983.90,1389.22,1415.96,1244.00,1189.42,1192.72,1702.48,1117.16,1244.04
Страховые - ПФР,183245.72,200529.62,169571.04,185389.81,159680.33,167392.77,196561.49,158489.42,166627.87,181162.29,...,183693.70,206928.96,166707.34,169914.92,149280.49,142730.11,143126.06,204297.96,139359.32,149285.51


In [164]:
view.df

,accepted,balance_by_tax,balance_by_type,credit,deadline,debit,decision_number,document_date,document_number,document_period,document_registered_date,document_type,kbk,operation_date,operation_details,payed_before_deadline,payment_type,tax
0,0.0,15642.00,15642.00,0.00,2020-01-01,0.00,NaN,NaT,,nan,NaT,nan,18210101012020000110,2020-01-01,переплата-сальдо начальное,0.0,Налог,Прибыль
16,0.0,-17408.52,-17408.52,0.00,2020-01-01,0.00,NaN,NaT,,nan,NaT,nan,18210202090070010160,2020-01-01,переплата-сальдо начальное,0.0,Пеня,Страховые - Нетрудоспособность и Материнство
14,0.0,-1328947.50,-97966.45,0.00,2020-01-01,0.00,NaN,NaT,,nan,NaT,nan,18210301000010000110,2020-01-01,переплата-сальдо начальное,0.0,Пеня,НДС
13,0.0,-97174.86,-79766.34,0.00,2020-01-01,0.00,NaN,NaT,,nan,NaT,nan,18210202090070010160,2020-01-01,переплата-сальдо начальное,0.0,Налог,Страховые - Нетрудоспособность и Материнство
12,0.0,-8600.56,-8600.56,0.00,2020-01-01,0.00,NaN,NaN,,nan,NaN,nan,18210202101080011160,2020-01-01,переплата-сальдо начальное,0.0,Пеня,Страховые - ОМС (до до 1 января 2017)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1724,0.0,-1613420.83,-419496.78,0.00,2022-11-01,298.48,NaN,NaT,,nan,NaT,nan,18210202101080013160,2022-11-04,начислены пени (по расчету),0.0,Пеня,Страховые - ОМС
1726,0.0,-32632.17,-9100.12,0.00,2022-11-01,5.90,NaN,NaT,,nan,NaT,nan,18210202132060010160,2022-11-04,начислены пени (по расчету),0.0,Пеня,Страховые - Доп. Тариф
1723,0.0,-6084096.40,-959824.18,0.00,2022-11-01,1104.32,NaN,NaT,,nan,NaT,nan,18210102010010000110,2022-11-04,начислены пени (по расчету),0.0,Пеня,НДФЛ
1727,0.0,-1615752.23,-1193867.70,56.35,2022-11-03,0.00,NaN,2022-11-01,2377,nan,2022-11-01,Платежное поручение,18210202101080013160,2022-11-09,уплачено,0.0,Налог,Страховые - ОМС


In [176]:
view.unidentified_insurance_payments()

,operation_date,tax,operation_details,credit,debit,decision_number,document_period,document_number
2058,2021-05-11,Страховые - ПФР,уплачено,55397.29,0.0,NaN,nan,856995
2059,2021-05-11,Страховые - ПФР,уплачено,22896.88,0.0,NaN,nan,856993
2070,2021-05-27,Страховые - ПФР,уплачено,49917.26,0.0,NaN,nan,43432
2071,2021-05-27,Страховые - ПФР,уплачено,45490.95,0.0,NaN,nan,43428
2072,2021-05-27,Страховые - ПФР,уплачено,28802.37,0.0,NaN,nan,43427
2073,2021-05-27,Страховые - ПФР,уплачено,20631.86,0.0,NaN,nan,43431
2076,2021-06-10,Страховые - ПФР,уплачено,27863.08,0.0,NaN,nan,129999
2078,2021-06-10,Страховые - ПФР,уплачено,694.66,0.0,NaN,nan,130001
2077,2021-06-10,Страховые - ПФР,уплачено,5849.35,0.0,NaN,nan,130004
2080,2021-06-18,Страховые - ПФР,уплачено,17849.85,0.0,NaN,nan,175129


In [180]:
view.insurance_overpayment()

,dec,payed_our,tax_report,ufns_tax_credit,ufns_penalty_credit,ufns_total_credit,bank_ver,delta,overpayment,document_numbers
2021-4,2137,1895901.0,1296130,1855344.53,34578.07,1889922.60,1895901.0,-5978.40,593792.60,NaN
2021-3,8785,2178329.0,1447603,1999045.15,38504.22,2037549.37,2178329.0,-140779.63,589946.37,NaN
2021-2,7310,2136134.0,1541730,2131953.06,4180.46,2136133.52,2136134.0,-0.48,594403.52,NaN
2021-1,4001,0.0,1538689,0.00,0.00,0.00,0.0,0.00,-1538689.00,NaN
2020-4,1854,2519235.9,1498729,1696165.58,24587.85,1720753.43,2519235.9,-798482.47,222024.43,NaN
2020-3,1133,2304460.0,1498729,2072327.05,26097.37,2098424.42,2304460.0,-206035.58,599695.42,NaN
2020-2,NaN,0.0,1595748,2286511.28,32960.07,2319471.35,0.0,2319471.35,723723.35,"[12021, 012023, 12023, 12026, 12029, 12025, 12..."


In [ ]:
view.df